In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait

import time
from datetime import datetime
from bs4 import BeautifulSoup

import cx_Oracle


# 데이터베이스 연결 설정
connection = cx_Oracle.connect('kyb/1111@localhost:1521/xe')
cursor = connection.cursor()

# 웹드라이버 설정
options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://map.naver.com/v5/search")
time.sleep(3)  # 페이지 로딩 대기

# 검색어 목록 가져오기
cursor.execute('SELECT id, search_name FROM zero') 
data_list = [(row[0], row[1]) for row in cursor.fetchall()]

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 현재 스크래핑된 HTML 요소 스냅샷 만드는 함수 (이거 있어야지 나중에 안될때 알아보기편함)
def save_html_snapshot(page_str, file_prefix):
    current_date = datetime.now().strftime('%y%m%d_%H%M%S')
    file_path = f'{file_prefix}_{current_date}.html'
    soup = BeautifulSoup(page_str, 'html.parser')
    pretty_html = soup.prettify()
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(pretty_html)

# 해당 요소 찾을 때까지 10초 대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
        return wait
    except Exception as e:
        print(f"{code} 태그를 찾지 못하였습니다. {e}")
        return None

# 각 검색어에 대해 작업 수행
for id, search_name in data_list:
    try:
        # 검색창 찾기 및 검색어 입력
        search_box = time_wait(10, 'div.input_box > input.input_search')
        if search_box:
            search_box.clear() #검색창 깨끗하게
            search_box.send_keys(search_name) #검색어 입력
            search_box.send_keys(Keys.ENTER) # 엔터누르기
            time.sleep(3)  # 검색 결과 로딩 대기

            # 검색 결과 프레임으로 전환 및 스냅샷 저장
            switch_frame('searchIframe')
            save_html_snapshot(driver.page_source, f"search_result_{id}") #저장이름

            # 정보 탭 클릭 및 프레임으로 전환 전 스냅샷 저장
            switch_frame('entryIframe')
            save_html_snapshot(driver.page_source, f"before_info_tab_click_{id}")

            info_tab_found = False
            try:
                tabs = driver.find_elements(By.XPATH, '//a[@role="tab"]//span') # 홈 리뷰 정보 등 탭있는곳 중에서
                for tab in tabs:
                    if tab.text == '정보':  #정보탭 찾기
                        driver.execute_script("arguments[0].scrollIntoView();", tab)
                        driver.execute_script("arguments[0].click();", tab)
                        info_tab_found = True
                        break
            except Exception as e:
                print(f"정보 탭을 찾지 못하였습니다: {e}")

            if not info_tab_found:
                print(f"정보 탭을 찾지 못하였습니다 for id: {id}")
                continue

            time.sleep(3)  # 정보 탭 로딩 대기
            save_html_snapshot(driver.page_source, f"info_{id}")

            # "펼쳐보기" 버튼 클릭
            more_buttons = driver.find_elements(By.CLASS_NAME, 'OWPIf')
            for btn in more_buttons:
                try:
                    driver.execute_script("arguments[0].click();", btn)
                    time.sleep(1)  # 버튼 클릭 후 로딩 대기
                except Exception as e:
                    print(f"Failed to click '펼쳐보기' for ID {id}: {e}")

            # 업데이트된 페이지 소스 가져오기
            updated_page_source = driver.page_source
            save_html_snapshot(updated_page_source, f"updated_info_{id}")

            # 정보와 해시태그 추출 및 데이터베이스 저장
            soup = BeautifulSoup(updated_page_source, 'html.parser')
            info = ''
            hash_tags = []

            info_element = soup.find("div", class_="T8RFa")
            hash_tag_elements = soup.find_all("span", class_="RLvZP")
            
            if info_element:
                info = info_element.get_text(separator=' ').strip()

            if hash_tag_elements:
                hash_tags = [element.get_text(separator=' ').strip() for element in hash_tag_elements]

            # 해시태그를 '#태그1, #태그2' 형식으로 변환
            hash_tags_str = '#{}'.format(', #'.join(hash_tags))

            # 디버깅: 추출된 정보와 해시태그 출력
            print(f"ID {id}: Extracted info and hash tags")
            print(f"Info: {info}")
            print(f"Hash Tags: {hash_tags_str}")

            # 데이터베이스에 정보와 해시태그 저장
            try:
                print("Binding variables for SQL query:")

                cursor.execute("""
                    UPDATE zero
                    SET info = :info, hash_tags = :hash_tags
                    WHERE id = :id
                """, {
                    'info': info,
                    'hash_tags': hash_tags_str,
                    'id': id
                })
                connection.commit()
                print(f"Successfully stored info and hash tags for ID {id} in the database")  # 데이터베이스 저장 성공 로그
            except cx_Oracle.DatabaseError as db_e:
                error, = db_e.args
                print(f"Database error occurred while storing info and hash tags for ID {id}: {error.message}")
                print(f"SQL Query: {cursor.statement}")
                print(f"Bound Variables: {cursor.bindvars()}")
            except Exception as e:
                print(f"Unexpected error occurred while storing info and hash tags for ID {id}: {e}")
    except Exception as e:
        print(f"Error occurred while processing search_name '{search_name}' with id '{id}': {e}")

    # 검색어 처리 후 브라우저 새로 고침
    driver.get("https://map.naver.com/v5/search")
    time.sleep(3)

# 드라이버 종료 및 연결 종료
driver.quit()
cursor.close()
connection.close()


ID 1: Extracted info and hash tags
Info: 더 나은 지구가 더 나은 우리를 만듭니다.
기후위기 시대에 살고 있는 우리와 다음세대를 위해 사람과 환경에 무해한 지속가능한 제품을 소개하고 리필스테이션, 자원순환소 운영, 플로깅 및 환경 강의 등 로컬 기반의 환경 캠페인을 도모합니다.
도시 재생과 지역의 상생을 목표로 더 좋은 지역,
더 좋은 지구를 만드는 데 선한 영향을 더하고자 합니다.
완벽한 소수보다 불완전한 다수가 함께 행동 할 수 있도록, 더 가까이, 더 쉽게, 더 나은 가치 소비의 삶을 실천하는 제로웨이스트 라이프스타일 편집샵.

취급품목: 친환경 생활용품, 세제류 리필
Hash Tags: #제로웨이스트, #편집샵, #친환경, #리필스테이션, #자원순환
Binding variables for SQL query:
Successfully stored info and hash tags for ID 1 in the database
ID 3: Extracted info and hash tags
Info: 건강한 지구 건강한 나의 삶을 지키기 위해
1.5 라이프스타일 속 제로웨이스트 샵 겸 카페 1.5도씨 입니다.
우리가 사는 동네에서 커피도 한잔하며 제로웨이스트를 알아가는
곳이며 우리의 일상을 공감하며 문화,예술 워크숍을 진행하는 곳이기도 합니다.

화장품 , 세제를 그람 당 필요한 만큼 담아갈 수 있는 리필스테이션
여러가지 식료품, 친환경 식자재 그로서리 리필스토어 입니다.

1.5도씨에서 1.5라이프스타일을 함께 하며
나의 건강한 삶을 만들고 자연과 동물 그리고 우리가 모두와
마음을 공감하며 지속 가능한 삶을 경험해 보세요~^^
Hash Tags: #
Binding variables for SQL query:
Successfully stored info and hash tags for ID 3 in the database
ID 4: Extracted info and hash tags
Info: 건강과 환경을 모두 지킬 수 있는
지